<a href="https://colab.research.google.com/github/spros1/HeartFailureAnalysis/blob/main/ExcerA_Heart_Failure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   Start the commands in order to load a file from Kaggle to Colab

---


In [2]:
pip install -q kaggle

In [3]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sprospap","key":"9a4854805a9655a32d9fa00b6ca4aa15"}'}

In [7]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [8]:
! chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d andrewmvd/heart-failure-clinical-data

  0% 0.00/3.97k [00:00<?, ?B/s]
100% 3.97k/3.97k [00:00<00:00, 7.84MB/s]


In [10]:
ls

heart-failure-clinical-data.zip  kaggle.json  sample_data/


In [11]:
! unzip heart-failure-clinical-data.zip

Archive:  heart-failure-clinical-data.zip
  inflating: heart_failure_clinical_records_dataset.csv  




2.   Check the data and import them to SqlDatabase

---




In [12]:
# Check dataset 
import pandas as pd
from google.colab import files
with open('heart_failure_clinical_records_dataset.csv', 'r') as data:
    df3 = pd.read_csv(data, encoding = ('ansi'))
    print (df3)

      age  anaemia  creatinine_phosphokinase  ...  smoking  time  DEATH_EVENT
0    75.0        0                       582  ...        0     4            1
1    55.0        0                      7861  ...        0     6            1
2    65.0        0                       146  ...        1     7            1
3    50.0        1                       111  ...        0     7            1
4    65.0        1                       160  ...        0     8            1
..    ...      ...                       ...  ...      ...   ...          ...
294  62.0        0                        61  ...        1   270            0
295  55.0        0                      1820  ...        0   271            0
296  45.0        0                      2060  ...        0   278            0
297  45.0        0                      2413  ...        1   280            0
298  50.0        0                       196  ...        1   285            0

[299 rows x 13 columns]


### Create an instance of Database and load to table the CSV

In [13]:
# create the SQLite Database
%pip install csv-to-sqlite

In [15]:
# import to table 
!csv-to-sqlite -f heart_failure_clinical_records_dataset.csv -t full -o db.sqlite

Written 299 rows into 1 tables in 0.016 seconds


### Accessing Database


In [16]:
#Accessing the SQL Database
import sqlite3

In [17]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [18]:
conn = create_connection("db.sqlite")

In [19]:
def select_top(conn, table,  n):
    """
    Query n first rows of the table
    :param conn: the Connection object
    :param table: The table to query
    :param n: Number of rows to query
    """
    cur = conn.cursor()
    cur.execute(f"SELECT * FROM [{table}] LIMIT :limitNum", {"limitNum": n})

    rows = cur.fetchall()
    return rows

In [20]:
def head(conn, table, n=300):
  rows = select_top(conn, table, n)
  for r in rows:
      print(r)

In [21]:
head(conn, 'heart_failure_clinical_records_dataset')

(75.0, 0, 582, 0, 20, 1, 265000.0, 1.9, 130, 1, 0, 4, 1)
(55.0, 0, 7861, 0, 38, 0, 263358.03, 1.1, 136, 1, 0, 6, 1)
(65.0, 0, 146, 0, 20, 0, 162000.0, 1.3, 129, 1, 1, 7, 1)
(50.0, 1, 111, 0, 20, 0, 210000.0, 1.9, 137, 1, 0, 7, 1)
(65.0, 1, 160, 1, 20, 0, 327000.0, 2.7, 116, 0, 0, 8, 1)
(90.0, 1, 47, 0, 40, 1, 204000.0, 2.1, 132, 1, 1, 8, 1)
(75.0, 1, 246, 0, 15, 0, 127000.0, 1.2, 137, 1, 0, 10, 1)
(60.0, 1, 315, 1, 60, 0, 454000.0, 1.1, 131, 1, 1, 10, 1)
(65.0, 0, 157, 0, 65, 0, 263358.03, 1.5, 138, 0, 0, 10, 1)
(80.0, 1, 123, 0, 35, 1, 388000.0, 9.4, 133, 1, 1, 10, 1)
(75.0, 1, 81, 0, 38, 1, 368000.0, 4.0, 131, 1, 1, 10, 1)
(62.0, 0, 231, 0, 25, 1, 253000.0, 0.9, 140, 1, 1, 10, 1)
(45.0, 1, 981, 0, 30, 0, 136000.0, 1.1, 137, 1, 0, 11, 1)
(50.0, 1, 168, 0, 38, 1, 276000.0, 1.1, 137, 1, 0, 11, 1)
(49.0, 1, 80, 0, 30, 1, 427000.0, 1.0, 138, 0, 0, 12, 0)
(82.0, 1, 379, 0, 50, 0, 47000.0, 1.3, 136, 1, 0, 13, 1)
(87.0, 1, 149, 0, 38, 0, 262000.0, 0.9, 140, 1, 0, 14, 1)
(45.0, 0, 582, 0, 14,